In [26]:
category = "LINK"

In [27]:
file_1 = "_rawdata.txt"
file_2 = "_price.txt"
file_3 = "_transfer.txt"
file_4 = "_account_distribution_count_1.txt"
file_5 = "_account_distribution_sum_1.txt"
file_6 = "_account_stats.txt"

In [28]:
file_path = category + file_1

In [29]:
import pandas as pd

data = pd.read_csv(file_path, sep="\t", header=None)
data.columns = ["TXID", "Date", "Unix Timestamp", "From Address", "To Address", "Amount"]
data['Unix Timestamp'] = pd.to_datetime(data['Unix Timestamp'], unit='s')

In [30]:
non_zero_transfer = data[data['Amount'] > 0]

In [31]:
DAU = non_zero_transfer[['Date', 'From Address']]
DAU = DAU.groupby("Date").size().reset_index(name='DAU')
DAU['Date'] = pd.to_datetime(DAU['Date'])

In [32]:
file_path_2 = category + file_2

price_data = pd.read_csv(file_path_2, sep="\t", header=None)
price_data.columns = ["Ticker", "Contract_Address", "DateTime", "OPEN", "HIGH", "LOW", "CLOSE", "Volume", "Market cap"]

In [33]:
tx_count = non_zero_transfer.groupby("Date").size().reset_index(name='Tx_count')
tx_count['Date'] = pd.to_datetime(tx_count['Date'])
tx_volume = price_data[['DateTime', 'CLOSE']]
tx_volume = tx_volume.iloc[1:]
tx_volume['DateTime'] = pd.to_datetime(tx_volume['DateTime'])
tx_volume.sort_values(by='DateTime', inplace=True)
tx_volume.columns=['Date', 'Price']
tx_volume['Price'] = pd.to_numeric(tx_volume['Price'])

In [34]:
target = price_data[['DateTime', 'Market cap']]
target = target.iloc[1:]
target['DateTime'] = pd.to_datetime(target['DateTime'])
target.columns = ['Date', 'Market Cap']
target['Market Cap'] = target['Market Cap'].replace('None', 0)
target['Market Cap'] = pd.to_numeric(target['Market Cap'])

In [35]:
price_data = price_data[['DateTime', 'Volume']]
price_data = price_data.iloc[1:]
price_data['DateTime'] = pd.to_datetime(price_data['DateTime'])
price_data.sort_values(by='DateTime', inplace=True)
price_data.columns=['Date', 'Volume']

In [36]:
d = pd.merge(DAU, price_data, how='inner')
d = pd.merge(d, tx_count, how='inner')
d['Volume(Token)'] = tx_count['Tx_count'] * tx_volume['Price']
d = pd.merge(d, tx_volume, how='inner')

In [37]:
d.Volume = pd.to_numeric(d.Volume)
d['Volume(Token)'] = pd.to_numeric(d['Volume(Token)'])

In [38]:
target = pd.merge(d[['Date']], target[['Date','Market Cap']])

In [39]:
re = d.corr(method='pearson')
re

,DAU,Volume,Tx_count,Volume(Token),Price
DAU,1.000000,0.269046,1.000000,0.802533,0.047426
Volume,0.269046,1.000000,0.269046,0.163356,0.587997
Tx_count,1.000000,0.269046,1.000000,0.802533,0.047426
Volume(Token),0.802533,0.163356,0.802533,1.000000,-0.029080
Price,0.047426,0.587997,0.047426,-0.029080,1.000000


In [40]:
d.fillna(0, inplace=True)

In [41]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
x_lst = d.columns.tolist()
x_lst.remove('Date')
x_lst.remove('Tx_count')
def make_result(x_lst, target_y):
    coef_lst = []
    score_lst = []
    for x in x_lst:
        X = d[x].values
        y = target[target_y].values
        X = X.reshape(len(X), 1)
        y = y.reshape(len(y), 1)
        regr = linear_model.LinearRegression()
        regr.fit(X, y)
        coef_lst.append(regr.coef_[0][0])
        score_lst.append(regr.score(X, y))
    return coef_lst, score_lst

coef_lst, score_lst = make_result(x_lst, 'Market Cap')
empty_lst = [0] * len(x_lst)
final = pd.DataFrame({'Coef' : empty_lst, 'Score' : empty_lst})
final['x_lst'] = x_lst
final.Coef = coef_lst
final.Score = score_lst
final = final.set_index('x_lst')
final.sort_values(by='Score', ascending=False, inplace=True)

In [42]:
top = final[final['Score'] > 0.5]

In [43]:
x_lst = ['Date']
x_lst.extend(top.index.tolist())
try:
    top = d[x_lst]
    top = top.set_index('Date')
    X = top
    y = target['Market Cap']
except KeyError:
    print("All features are useless to predict Market Cap (= R2 score less than 0.5)")

try:
    if re.iloc[0][1] > 0.85:
        from sklearn.decomposition import PCA
        pca = PCA(n_components=1)
        model_pca = pca.fit(top.values)
        model_pca.transform(top.values)
        from sklearn.cross_decomposition import PLSRegression
        reg = PLSRegression(n_components=1)
        reg.fit(X, y)
        print("This Token's linear regression has done by : ", x_lst[1:])
        print("Used pls regression, R score is : ", reg.score(X, y))
    else:
        from sklearn.linear_model import LinearRegression
        reg = LinearRegression().fit(X, y)
        print("This Token's linear regression has done by : ", x_lst[1:])
        print("Used ols regression, R score is : ", reg.score(X, y))
except ValueError:
    print("")

This Token's linear regression has done by :  ['Price']
Used ols regression, R score is :  0.9999999175365564


# model = reg.predict(예측값)